#Цель работы: рассчет количества созданных карточек за определенный промежуток времени

In [1]:
import pandas as pd
import re

Подгружаем необходимые данные:
- выгрузка всех карточек сотрудников;
- данные по региональным лидера и супервайзерам.

In [3]:
user_team = pd.read_csv('/content/drive/MyDrive/Work/users-team (12).csv', encoding="utf-8", delimiter=';')
RL_and_SV = pd.read_csv('/content/drive/MyDrive/Work/Список СВ и РЛ.csv', encoding="utf-8", delimiter=';')

Переименовываем колонки для удобства использования

In [4]:
user_team.columns = [name.replace(' ','_') for name in user_team.columns]

In [5]:
RL_and_SV = RL_and_SV.rename(columns={'СВ':'Руководитель'})

Удаляем дубликаты карточек сотрудников по столбцу "СНИЛС"

In [6]:
user_team = user_team.drop_duplicates('СНИЛС')

Меняем формат столба "Дата_создания_пользователя" в дату

In [7]:
user_team.Дата_создания_пользователя = pd.to_datetime(user_team.Дата_создания_пользователя, format='%d-%m-%Y')

Убираем лишние столбы

In [8]:
user_team = user_team.drop(['Контрагент', 'Домашний_телефон_исполнителя', 'email', 'Активен', 'ID_руководителя', 'СНИЛС_руководителя', 'Роль_руководителя', 'Дата_последнего_визита'], axis=1)

Проходимся циклом и убираем все карточки, которые имеют подписи Farma и Vacant  
Чистим столбы на возможные двойные пробелы

In [9]:
for i in user_team.Пользователь:
    if re.search(r'[Farma]|[Vacant]', i):
        user_team = user_team[user_team.Пользователь != i ]
user_team = user_team.dropna(subset=['Руководитель'])
user_team.Руководитель = [i.replace('  ',' ') for i in user_team.Руководитель]

Создаем функцию для рассчета новых карточек сотрудников по необходимым датам

In [10]:
def search_person_for_date(dt1, dt2, name):
    df = user_team[(user_team.Дата_создания_пользователя >= dt1) & (user_team.Дата_создания_пользователя <= dt2)]
    df = df.merge(RL_and_SV, how='left', on=['Руководитель']).dropna(subset=['РЛ'])
    df = df.groupby(['Проект', 'РЛ'], as_index=False).agg({'Пользователь':'count'}).rename(columns={'Пользователь':name})
    return df

Создаем переменные - необходимые даты

In [11]:
febrary = search_person_for_date('02/01/2023', '02/28/2023', 'Февраль')
march = search_person_for_date('03/01/2023', '03/31/2023', 'Март')
april = search_person_for_date('04/01/2023', '04/30/2023', 'Апрель')
april_first = search_person_for_date('04/01/2023', '04/09/2023', '01/04/23 - 09/04/23')
april_second = search_person_for_date('04/10/2023', '04/16/2023', '10/04/23 - 16/04/23')
april_third = search_person_for_date('04/17/2023', '04/23/2023', '17/04/23 - 23/04/23')
april_four = search_person_for_date('04/24/2023', '04/30/2023', '24/04/23 - 30/04/23')

Склеиваем все полученные данные

In [19]:
total = febrary.merge(march, how='outer', on=['Проект', 'РЛ'])\
                       .merge(april, how='outer', on=['Проект', 'РЛ'])\
                       .merge(april_first, how='outer', on=['Проект', 'РЛ'])\
                       .merge(april_second, how='outer', on=['Проект', 'РЛ'])\
                       .merge(april_third, how='outer', on=['Проект', 'РЛ'])\
                       .merge(april_four, how='outer', on=['Проект', 'РЛ']).fillna(0)

Результат

In [20]:
total

,Проект,РЛ,Февраль,Март,Апрель,01/04/23 - 09/04/23,10/04/23 - 16/04/23,17/04/23 - 23/04/23,24/04/23 - 30/04/23
0,Corp,RL Семенова Татьяна,19,18,26,8,8.0,7,3
1,Corp,RL-Булатов Дмитрий Леонтьевич,25,40,35,12,10.0,7,6
2,Corp,RL-Бутримов Алексей Борисович,32,26,45,11,8.0,13,13
3,Corp,RL-Дрожжа Елизавета Александровна,23,26,22,8,4.0,3,7
4,Corp,RL-Казаров Александр Вячеславович,22,32,25,6,5.0,6,8
5,Corp,RL-Кашицин Виталий Сергеевич,21,17,37,11,10.0,8,8
6,Corp,RL-Логачев Анатолий Юрьевич,25,30,19,9,3.0,5,2
7,Corp,RL-Медведева Юлия Викторовна,24,20,40,13,10.0,9,8
8,Gill,RL Мартыненко Ирина,7,10,11,6,1.0,1,3
9,Gill,RL Сокольских Ольга,14,9,17,5,0.0,9,3


Рассчитываем итоги по каждому проекту для отправления в тексте письма

In [ ]:
itog = total.groupby('Проект', as_index=False).agg({'Март':'sum',	'Апрель':'sum',	'01/04/23 - 09/04/23':'sum',	'10/04/23 - 16/04/23':'sum', '17/04/23 - 23/04/23':'sum', '24/04/23 - 30/04/23':'sum'})

In [ ]:
itog

,Проект,Март,Апрель,01/04/23 - 09/04/23,10/04/23 - 16/04/23,17/04/23 - 23/04/23,24/04/23 - 30/04/23
0,Corp,209,249,78,58.0,58,55
1,Gill,131,179,48,41.0,43,47


Записываем данные в Excel

In [ ]:
with pd.ExcelWriter('/content/drive/MyDrive/Work/Users_for_RL.xlsx', engine = 'openpyxl') as writer:  
    total.to_excel(writer, sheet_name = 'Count', index = False)
    itog.to_excel(writer, sheet_name = 'itog', index = False)